From a slack message by L Grey in the `servicex-cms` chat.

In [12]:
from coffea.nanoevents import NanoEventsFactory, NanoAODSchema
from dask.distributed import Client
import dask
import dask_awkward
import awkward as ak
import hist.dask as hda

def extract_pushdown(coll):
    hlg_sorted = coll.dask._toposort_layers()
    pushdown_deps = []
    for key in hlg_sorted:
        annotations = coll.dask.layers[key].annotations
        if annotations is not None and "pushdown" in annotations:
            #print(key, coll.dask.layers[key].annotations)
            pushdown_deps = [key] + pushdown_deps
    for dep in pushdown_deps:
        layer = coll.dask.layers[dep]
        fcn = list(layer.dsk.values())[0][0]
        if isinstance(layer, dask_awkward.layers.AwkwardBlockwiseLayer) and not isinstance(layer, dask_awkward.layers.AwkwardInputLayer):
            print(layer.dsk)
            # print(dir(layer))
            print(list(layer.keys()))
            print(dep, fcn.fn)
            # print(dir(fcn))
            # print(fcn.arg_repackers[0])
            print("")
        else:
            print(dep, fcn)

if __name__ == "__main__":
    #client = Client()


    dask.config.set({"awkward.optimization.enabled": True, "awkward.raise-failed-meta": True, "awkward.optimization.on-fail": "raise"})

    with dask.annotate(pushdown="servicex"):
        events = NanoEventsFactory.from_root(
            {
                "nano_dy.root": "Events",
                "nano_dy.root": "Events",
                "nano_dy.root": "Events",
                "nano_dy.root": "Events",
            },
            metadata={"dataset": "nano_dy"},
            schemaclass=NanoAODSchema,
        ).events()

        mask = events.Muon.pt > 30
        events = events[ak.any(mask, axis=1)]
        
    myhist = hda.Hist.new.Regular(50, 0, 2.5, name="abseta").Double()

    myhist.fill(abseta=abs(events.Muon.eta))

    extract_pushdown(myhist)

{'getitem-3c0684871d892683f9c6b23773296e63': (<dask_awkward.lib.core.ArgsKwargsPackedFunction object at 0x0000026DE34308D0>, '__dask_blockwise__0', '__dask_blockwise__1')}
[('getitem-3c0684871d892683f9c6b23773296e63', 0)]
getitem-3c0684871d892683f9c6b23773296e63 <built-in function getitem>

{'any-69bfecac024bf65a681cb988c441f351': (<dask_awkward.lib.core.ArgsKwargsPackedFunction object at 0x0000026DE3B0D910>, '__dask_blockwise__0')}
[('any-69bfecac024bf65a681cb988c441f351', 0)]
any-69bfecac024bf65a681cb988c441f351 <function any at 0x0000026DB2A03F60>

{'greater-12d3cff3091a1bf2a22e55f34f6a120e': (<dask_awkward.lib.core.ArgsKwargsPackedFunction object at 0x0000026DE3A166D0>, '__dask_blockwise__0', '__dask_blockwise__1')}
[('greater-12d3cff3091a1bf2a22e55f34f6a120e', 0)]
greater-12d3cff3091a1bf2a22e55f34f6a120e <ufunc 'greater'>

{'pt-6f6350d8daa3ad840e45076c81c732a6': (<dask_awkward.lib.core.ArgsKwargsPackedFunction object at 0x0000026DE1635A10>, '__dask_blockwise__0', '__dask_blockwise